In [7]:
import os
import pandas as pd


df=pd.read_csv("https://data.heatonresearch.com/data/t81-558/auto-mpg.csv",
               na_values=['NA','?'])
df.insert(1,'weight_kg',(df['weight'] * 0.45359237).astype(int))
display(df)

,mpg,weight_kg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
0,18.0,1589,8,307.0,130.0,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,1675,8,350.0,165.0,3693,11.5,70,1,buick skylark 320
2,18.0,1558,8,318.0,150.0,3436,11.0,70,1,plymouth satellite
3,16.0,1557,8,304.0,150.0,3433,12.0,70,1,amc rebel sst
4,17.0,1564,8,302.0,140.0,3449,10.5,70,1,ford torino
...,...,...,...,...,...,...,...,...,...,...
393,27.0,1265,4,140.0,86.0,2790,15.6,82,1,ford mustang gl
394,44.0,966,4,97.0,52.0,2130,24.6,82,2,vw pickup
395,32.0,1040,4,135.0,84.0,2295,11.6,82,1,dodge rampage
396,28.0,1190,4,120.0,79.0,2625,18.6,82,1,ford ranger


In [10]:
if 'GOOGLE_API_KEY' in os.environ:
    # If the API key is defined in an environmental variable,
    # the use the env variable.
    GOOGLE_KEY = os.environ['GOOGLE_API_KEY']    
else:
    # If you have a Google API key of your own, you can also just
    # put it here:
    GOOGLE_KEY = ''
display(GOOGLE_KEY)

In [12]:
import requests

address = "5057 Woodward Ave, Detroit, MI 48202"

response = requests.get(
    'https://maps.googleapis.com/maps/api/geocode/json?key={}&address={}' \
    .format(GOOGLE_KEY,address))

resp_json_payload = response.json()

if 'error_message' in resp_json_payload:
    print(resp_json_payload['error_message'])
else:
    print(resp_json_payload['results'][0]['geometry']['location'])

{'lat': 42.35742339999999, 'lng': -83.06534219999999}


#### Great circle distance

$\Delta\sigma= \arccos (\sin\phi_1 . \sin\phi_1 + \cos\phi_1 . \cos\phi_2 . \cos(\Delta\lambda))$


$d = r \delta\sigma$

In [14]:
from math import sin,cos,sqrt,atan2, radians

URL='https://maps.googleapis.com' + \
    '/maps/api/geocode/json?key={}&address={}'

def distance_lat_lng(lat1,lng1,lat2,lng2):
    # approximate radius of earth in km
    R=6373.0

    lat1=radians(lat1)
    lng1=radians(lng1)
    lat2=radians(lat2)
    lng2=radians(lng2)

    dlng=lng2-lng1
    dlat=lat2-lat1

    a=sin(dlat/2)**2+cos(lat1)*cos(lat2)*sin(dlng/2)**2
    c=2*atan2(sqrt(a),sqrt(1-a))

    return R*c

# Find lat lon for address
def lookup_lat_lng(address):
    response = requests.get( \
        URL.format(GOOGLE_KEY,address))
    json = response.json()
    if len(json['results']) == 0:
        raise ValueError("Google API error on: {}".format(address))
    map = json['results'][0]['geometry']['location']
    return map['lat'],map['lng']

# Distance between two locations

import requests

address1 = "5057 Woodward Ave, Detroit, MI 48202" 
address2 = "404 Westwood Plaza Suite 277, Los Angeles, CA 90095"

lat1, lng1 = lookup_lat_lng(address1)
lat2, lng2 = lookup_lat_lng(address2)

print("Distance, WSU, MI to UCLA, CA: {} km".format(
        distance_lat_lng(lat1,lng1,lat2,lng2)))


Distance, WSU, MI to UCLA, CA: 3200.5555611923874 km


In [15]:
# Encoding other universities by their distance to Washington University

schools = [
    ["Princeton University, Princeton, NJ 08544", 'Princeton'],
    ["Massachusetts Hall, Cambridge, MA 02138", 'Harvard'],
    ["5801 S Ellis Ave, Chicago, IL 60637", 'University of Chicago'],
    ["Yale, New Haven, CT 06520", 'Yale'],
    ["116th St & Broadway, New York, NY 10027", 'Columbia University'],
    ["450 Serra Mall, Stanford, CA 94305", 'Stanford'],
    ["77 Massachusetts Ave, Cambridge, MA 02139", 'MIT'],
    ["Duke University, Durham, NC 27708", 'Duke University'],
    ["University of Pennsylvania, Philadelphia, PA 19104", 
         'University of Pennsylvania'],
    ["Johns Hopkins University, Baltimore, MD 21218", 'Johns Hopkins']
]

lat1, lng1 = lookup_lat_lng("5057 Woodward Ave, Detroit, MI 48202")

for address, name in schools:
    lat2,lng2 = lookup_lat_lng(address)
    dist = distance_lat_lng(lat1,lng1,lat2,lng2)
    print("School '{}', distance to WSU is: {}".format(name,dist))

School 'Princeton', distance to WSU is: 736.4607287858638
School 'Harvard', distance to WSU is: 981.0359140625056
School 'University of Chicago', distance to WSU is: 379.71866750326893
School 'Yale', distance to WSU is: 847.9806534298589
School 'Columbia University', distance to WSU is: 778.5171737120637
School 'Stanford', distance to WSU is: 3351.725927242082
School 'MIT', distance to WSU is: 983.1807419018292
School 'Duke University', distance to WSU is: 791.2227334892037
School 'University of Pennsylvania', distance to WSU is: 711.1379635361969
School 'Johns Hopkins', distance to WSU is: 638.0718554963561
